In [43]:
%matplotlib inline
from __future__ import division
import sys
if '../' not in sys.path: sys.path.append("../")

import matplotlib.pyplot as plt
    
import numpy as np
from amnet import Variable, Linear, Constant
from amnet import atoms, lyap
import amnet
from amnet.util import mfp
import z3
# set up global z3 parameters
# parameters from https://stackoverflow.com/a/12516269
z3.set_param('auto_config', False)
z3.set_param('smt.case_split', 5)
z3.set_param('smt.relevancy', 2)



In [44]:
def fn_dynamics(x,u):
    A = np.array([[1,-1],[0,1]])
    #A = np.array([[1,-1],[-0.4,1]])
    B = np.array([[0], [1]])
    f = A*x+B*u
    return f
   
    
n = 2
m = 1
T = 5

x_cont = Variable(outdim=T*(n+m),name='XCont')
x = []
u = []
for t in xrange(T):
    x.append(x_cont[t*(n+m)+0:t*(n+m)+n])
    u.append(x_cont[t*(n+m)+n:t*(n+m)+n+m])
    assert len(x[t]) == n and len(u[t])==m
    
x_init = Constant(x[0],np.array([[0],[0]]))
x_final = Constant(x[T-1],np.array([[100],[0]]))
u_sat = 5
u_min = 1
    
obj = amnet.opt.Minimize(amnet.atoms.norm1(amnet.atoms.from_list(u)))
cons = []
cons.append(x[0] == x_init)
for t in xrange(T-1):
    cons.append(x[t+1] == fn_dynamics(x[t],u[t]))
    cons.append(amnet.atoms.norminf(u[t]) <= u_sat)
    #cons.append(amnet.atoms.norminf(u[t]) >= u_min)
cons.append(amnet.atoms.norminf(u[T-1]) <= u_sat)
#cons.append(amnet.atoms.norminf(u[T-1]) >= u_min)
cons.append(x[T-1] == x_final)
Opts = amnet.opt.OptOptions()
Opts.fptol = float(2**-5)
prob = amnet.opt.Problem(obj,cons,options=Opts)
result = prob.solve()

OptResult(objval=None, value=None, status=<OptResultCode.INFEASIBLE: 3>, model=None)


In [41]:
xv = []
uv = []
for t in xrange(T):
    xv.append(x[t].eval(result.value))
    uv.append(u[t].eval(result.value))

x1plot,x2plot = zip(*xv)
fig, ax = plt.subplots(1,2,figsize=(10,5))
ax[0].plot(x1plot,x2plot,'x-b')
ax[0].set_title('System trajectory')
ax[0].set_xlim(-5,10)
ax[0].set_ylim(-5,10)
uplot = zip(*uv)
ax[1].plot(xrange(T),np.array(uplot).T,'xr')
ax[1].set_xlim(0,T)
ax[1].set_ylim(-10,10)
ax[1].set_title('Applied control $u(t)$')
plt.show()
print uv

TypeError: object of type 'NoneType' has no len()